# Unbinned log likelihood fitting with nllfitter


This is an example of estimating model parameters of a probability distribution function by minimizing the negative log likelihood (nll).  This will be carried out on dataset that was generated to simulate the discovery of the Higgs boson in the diphoton decay channel.  This can all be run via the script (test/hgg_fit.py).  Let's first import all the necessary modules:

In [1]:
#imports and config
%cd '/home/naodell/work/CMS/amumu'
%matplotlib notebook

import numpy as np
import pandas as pd
from scipy.stats import norm
from numpy.polynomial.legendre import legval
from lmfit import Parameters

from nllfitter import NLLFitter, Model
from nllfitter.fit_tools import get_data, fit_plot_1D, scale_data
%matplotlib inline

/home/naodell/work/CMS/amumu


Now we'll need get the data that we will be modelling.  Given that the Higgs boson is known to have a mass of 125 GeV, we will only consider datapoints lying between 100 and 180 GeV.  The get_data function will scale the data to lie in the range [-1, 1].  This is motivated by the fact that we will be using Legendre polynomials to model the background.

In [2]:
xlimits = (100., 180.)
data = pd.read_csv('data/toy_hgammagamma.txt')
data = data.diphoton_mass.values
n_total = data.size

Let's define the background pdf and background plus signal datasets.

In [6]:
def bg_pdf(x, a):
    '''
    Second order Legendre Polynomial with constant term set to 0.5.

    Parameters:
    ===========
    x: data
    a: model parameters (a1 and a2)
    '''
    z   = scale_data(x, xmin=100, xmax=180)
    fx  = legval(z, [0.5, a[0], a[1], a[2]])*2/(100 - 180)
    return fx

def sig_pdf(x, a):
    '''
    Third order Legendre Polynomial (normalized to unity) plus a Gaussian.

    Parameters:
    ===========
    x: data
    a: model parameters (a1, a2, mu, and sigma)
    '''
    return (1 - a[0])*bg_pdf(x, a[3:6]) + a[0]*norm.pdf(x, a[1], a[2])


For the background model (null hypothesis) we will use a third order Legendre polynomial normalized to unity (this is satisfied by requiring the coefficient for the constand term be 0.5).  The background plus signal model (alternative model) will include an additional Gaussian term.  The mean of the Gaussian should correspond to the Higgs boson mass.

Let's set up the background only fit,

In [7]:
bg_params = Parameters()
bg_params.add_many(
                   ('a1', 0., True, None, None, None),
                   ('a2', 0., True, None, None, None),
                   ('a3', 0., True, None, None, None)
                  )

bg_model  = Model(bg_pdf, bg_params)
bg_fitter = NLLFitter(bg_model)

The NLLFitter framework uses the lmfit Parameters class.  The usage is described [here](https://lmfit.github.io/lmfit-py/parameters.html).  The Model class holds the functional form of the pdf and the parameters.  The NLLFitter is initialized with bg_model which will be used to fit to the data as follows, 

In [8]:
bg_result = bg_fitter.fit(data)

Fit finished with status: 9


That's all there is to it.  (N.B. the parameters are correct for the range [-1, 1], but in general won't be for the true range.  This will be addressed in iterations in the near future).  Let's do the same for the background plus signal model,

In [9]:
sig_params = Parameters()
sig_params.add_many(
                    ('A'     , 0.01 , True , 0.   , 1.   , None),
                    ('mu'    , -0.3 , True , -0.8 , 0.8  , None),
                    ('sigma' , 0.01 , True , 0.01 , 1.   , None),
                   )
sig_params += bg_params.copy()
sig_model  = Model(sig_pdf, sig_params)
sig_fitter = NLLFitter(sig_model)
sig_result = sig_fitter.fit(data)

Fit finished with status: 9


Note that I was able to just add a copy of the background parameters to the signal parameters.  Be careful to pass a copy, though, or otherwise the values in bg_params will be different after carrying out the fit of sig_model to the data.  Let's see how our fits look,

In [ ]:
fit_plot(data, xlimits, sig_model, bg_model, 'hgg', show=True)

Thats' a damn fine fit there.  By eye it looks pretty clear that the background plus signal fits to the data better.  The standard figure of merit for quantizing this model comparison is the likelihood ratio vis-a-vis Wilk's theorem,

In [ ]:
q = 2*(bg_model.calc_nll(data) - sig_model.calc_nll(data))
print 'z = {0}'.format(np.sqrt(q))

Given the pseudodata was generated by injecting $\sim 5\sigma$ signal, this result is to be expected.